In [6]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory

# Chemin vers ton dataset
data_path = "/Users/mac/Desktop/Projet_MLOPS/fastapi_streamlit-main/photo/"


# Charger les datasets avec split train/validation
train_dataset = image_dataset_from_directory(
    data_path,
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=(224, 224),
    batch_size=32
)

val_dataset = image_dataset_from_directory(
    data_path,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=(224, 224),
    batch_size=32
)

# Normalisation des pixels
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_dataset = train_dataset.map(lambda x, y: (normalization_layer(x), y))
val_dataset = val_dataset.map(lambda x, y: (normalization_layer(x), y))


Found 2527 files belonging to 6 classes.
Using 2022 files for training.
Found 2527 files belonging to 6 classes.
Using 505 files for validation.


In [7]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model

# Charger MobileNetV2 sans sa tête (couche de classification)
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

# Geler les couches du modèle pré-entraîné
base_model.trainable = False

# Ajouter des couches personnalisées pour la classification
x = base_model.output  # Sortie du modèle pré-entraîné
x = GlobalAveragePooling2D()(x)  # Pooling global
x = Dense(128, activation="relu")(x)  # Couche dense
x = Dropout(0.5)(x)  # Dropout pour réduire le surapprentissage
output = Dense(6, activation="softmax")(x)  # 6 classes pour la classification

# Créer le modèle complet
model = Model(inputs=base_model.input, outputs=output)


In [9]:
from tensorflow.keras.optimizers import Adam

# Compiler le modèle
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

# Afficher le résumé du modèle pour vérifier la structure
model.summary()



Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer_1[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 2,422,726 (9.24 MB)

 Trainable params: 164,742 (643.52 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [10]:
# Entraîner le modèle
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=10,
    verbose=1
)


Epoch 1/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 21s 296ms/step - accuracy: 0.2784 - loss: 1.8983 - val_accuracy: 0.5921 - val_loss: 1.2235
Epoch 2/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 19s 290ms/step - accuracy: 0.5312 - loss: 1.2536 - val_accuracy: 0.6792 - val_loss: 0.9769
Epoch 3/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 19s 302ms/step - accuracy: 0.6319 - loss: 1.0221 - val_accuracy: 0.6990 - val_loss: 0.8545
Epoch 4/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 21s 330ms/step - accuracy: 0.6688 - loss: 0.8636 - val_accuracy: 0.7426 - val_loss: 0.7696
Epoch 5/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 20s 308ms/step - accuracy: 0.7208 - loss: 0.7670 - val_accuracy: 0.7505 - val_loss: 0.7191
Epoch 6/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 20s 308ms/step - accuracy: 0.7444 - loss: 0.7372 - val_accuracy: 0.7723 - val_loss: 0.6844
Epoch 7/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 20s 304ms/step - accuracy: 0.7458 - loss: 0.6806 - val_accuracy: 0.7723 - val_loss: 0.6463
Epoch 8/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 20s 310ms/step - accuracy: 0.7959 - loss: 0.6071 - val_accu

In [11]:
# Évaluation sur le dataset de validation
val_loss, val_accuracy = model.evaluate(val_dataset)
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy}")


16/16 ━━━━━━━━━━━━━━━━━━━━ 4s 245ms/step - accuracy: 0.7826 - loss: 0.6214
Validation Loss: 0.5878581404685974
Validation Accuracy: 0.7940593957901001


In [12]:
# Sauvegarder le modèle
model.save("model.h5")
print("Modèle sauvegardé sous le nom 'model.h5'")


Modèle sauvegardé sous le nom 'model.h5'
